In [1]:
#Importing Libraries
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import gurobipy as gp
from gurobipy import GRB

In [9]:
prices_df = pd.read_csv('prices_merged_df_output.csv', delimiter=',')

balUPVec = np.array(prices_df['BalancingPriceUpEUR'].values)
balDOWNVec = np.array(prices_df['BalancingPriceDownEUR'].values)
DAVec = np.array(prices_df['DA_PriceEUR'].values)

features_df = pd.read_csv('merged_df_output.csv', delimiter=',')


actualWindproduction = np.array(features_df[' Total Wind Production Active Power'].values)
print(actualWindproduction)

power_pred=[3560.02,2800.32,4500.65,3200.90,2100.76]


[3520.38803673 2820.53997898 3302.27914619 ... 3928.63138103 3481.61078453
 3848.66349792]


In [7]:
#Setting Parameters
hourblock=[hour for hour in range(1,5)]

In [13]:
model=gp.Model()
#1. Decision Variable
power_pred=model.addVars(
    hourblock, vtype=GRB.INTEGER, name="power_pred"
)
#2. Constraints
# C= The total capacity of the wind farm
model.addConstrs(
    (power_pred[t]<=30000)
    for t in range(hourblock)
)
#3. Objective Function
model.setObjective(
    # The revenue obtained from the day-ahead market
    gp.quicksum(
        DAVec[t]*power_pred[t]
        for t in range(hourblock)
    )+
    #The revenue obtained from the balancing market
    gp.quicksum(
        balDOWNVec[t]*max(0,actualWindproduction[t]-power_pred[t]) #max() function will eliminate the negative value in the function
        for t in range(hourblock)
    )-
    gp.quicksum(
        balUPVec[t]*max(0,power_pred[t]-actualWindproduction[t])
        for t in range(hourblock)
    )
,GRB.MAXIMIZE)
model.optimize()

TypeError: 'list' object cannot be interpreted as an integer